Ссылка на репозиторий проекта:
https://github.com/andrew-tvorogov/practicum_masterskaya_2

# Маркетинг. Предсказание вероятности покупки пользователем в интернет-магазине в течение 90 дней.

<b>Описание:</b>
- Интернет-магазин собирает историю покупателей, проводит рассылки предложений и планирует будущие продажи. Для оптимизации процессов надо выделить пользователей, которые готовы совершить покупку в ближайшее время.

<b>Заказчик исследования:</b>
- yandex practicum

<b>Источник данных:</b>
- yandex practicum

<b>Содержание данных:</b>
- история покупок;
- история рекламных рассылок.

<b>Цель:</b>
- Предсказать вероятность покупки в течение 90 дней

<b>Задачи:</b>
- Изучить данные
- Разработать полезные признаки
- Создать модель для классификации пользователей
- Улучшить модель и максимизировать метрику roc_auc
- Выполнить тестирование

<b>Данные доступны в трёх файлах:</b>

- ``apparel-purchases.csv`` - история покупок
    - ``client_id`` - идентификатор пользователя
    - ``quantity`` - количество товаров в заказе
    - ``price`` - цена товара
    - ``category_ids`` - вложенные категории, к которым отнсится товар
    - ``date`` - дата покупки
    - ``message_id`` - идентификатор сообщения из рассылки

- ``apparel-messages.csv`` - история рекламных рассылок
    - ``bulk_campaign_id`` - идентификатор рекламной кампании
    - ``client_id`` - идентификатор пользователя
    - ``message_id`` - идентификатор сообщений
    - ``event`` - тип действия
    - ``channel`` - канал рассылки
    - ``date`` - дата рассылки
    - ``created_at`` - точное время создания сообщения

- ``apparel-target_binary`` - целевой бинарный признак, совершит или не совершит клиент покупку в течение следующих 90 дней
    - ``client_id`` - идентификатор пользователя
    - ``target`` - целевой признак

In [2]:
# установка пакетов

# correlation analyzer library
!pip install phik --quiet 

In [3]:
# Загрузка библиотек
import pandas as pd
import matplotlib.pyplot as plt # для диаграмм
import math # для диаграмм
import numpy as np # для равномерного распределения значений на диаграмме
import seaborn as sns
import phik

from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler,
    MinMaxScaler,
    RobustScaler, 
    OneHotEncoder, 
    OrdinalEncoder)

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, f1_score

from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectKBest, f_classif

from warnings import simplefilter

In [4]:
# Функции

def dataset_info(df):
    '''
    Покажет сводную информацию по датасету
    '''
    cat_columns = df.select_dtypes('object').columns # категориальные
    num_columns = df.select_dtypes('float').columns # количественные
    datetime_columns = df.select_dtypes('datetime64[ns]').columns # датавремя
    print(f'''\n''')
    df.info()
    print(f'''\n''')
    display(df.describe())
    if (len(list(datetime_columns)) > 0):
        display(startups_data[datetime_columns].agg(['min','max']))
        print(f'''\n''')
    print(f'''\n''')
    display(df.head(3))
    print(f'''\n\nЯвных дублей: {df.duplicated().sum()}\n\n''')
    print(f'''Пропущенных значений:\n{df.isna().sum()}\n\n''')

def dataset_tm_info(df):
    '''
    Покажет информацию по datetime колонкам датасета
    '''
    datetime_columns = df.select_dtypes('datetime64[ns]').columns # датавремя
    if (len(list(datetime_columns)) > 0):
        display(startups_data[datetime_columns].agg(['min','max']))
        #print(f'''\n''')
    else:
        print(f'''Нет колонок типа datetime\n''')

# функция для отображения круговой диаграммы распределения количества элементов по категории
def show_category_pie(df, field, title):
    '''
    Отобразит круговую диаграмму распределения количества элементов по категории
    '''
    df[field]\
    .value_counts().plot(kind='pie', title = title + ', %', autopct='%1.0f%%', figsize=(5,5))
    plt.ylabel('')
    plt.show()

# функция для отображения количественных признаков в виде графиков - гистограммы и ящика с усами
def show_numeric_bplt_hist(df, variable, first_label, second_label, bins):
    '''
    Отобразит количественные признаки в виде графиков - гистограммы и ящика с усами
    '''
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.hist(df[variable], bins=bins)
    ax1.set_title('Гистограмма')
    ax1.set_xlabel(first_label)
    ax1.set_ylabel(second_label)    
    ax1.grid(True)    
    
    ax2.boxplot(df[variable], vert=False)
    ax2.set_title('Ящик с усами')
    ax2.set_xlabel(first_label)
    ax2.set_ylabel(second_label)
    ax2.grid(True)
    ax2.set_yticks([])

    plt.show()
    
def show_two_barh(df_left, df_right, field, y_label, left_title, right_title):
    '''
    Отобразит две столбчатые диаграммы(горизонтальные) распределения количества элементов по категории для двух датасетов
    '''
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

    df_left[field].value_counts(normalize=True).sort_values().plot.barh(alpha=0.5, color=colors[0], label='', ax=ax1)
    ax1.set_title(left_title)
    ax1.set_ylabel(y_label)    
    df_right[field].value_counts(normalize=True).sort_values().plot.barh(alpha=0.5, color=colors[1], label='', ax=ax2)    
    ax2.set_title(right_title)
    plt.show()
        
def show_two_countplots(df_left, df_right, field, y_label, x_label, left_title, right_title, y_lim=(0, 100), rotation=0, sort_field=None, prefix=''):
    '''
    Отобразит две countplot диаграммы распределения количества элементов по категории для двух датасетов
    '''
    #if (sort_field):
    #    df_left['day_index'] = df_left[sort_field] # индекс для сортировки
    #    df_left_copy = df_left.sort_values(by=f'{prefix}_{sort_field}')[field].copy()
    #    display(df_left_copy)     
    
    fig, axes = plt.subplots(1, 2)
    fig.set_size_inches(16, 4)
    
    left = sns.countplot(df_left[field], color=colors[0], ax=axes[0], alpha=0.5)
    left.set(xlabel=x_label, ylabel=y_label, title=left_title, ylim=y_lim)
    left.set_xticklabels(left.get_xticklabels(), rotation=rotation)
    
    right = sns.countplot(df_right[field], color=colors[1], ax=axes[1], alpha=0.5)
    right.set(xlabel=x_label, ylabel=y_label, title=right_title, ylim=y_lim)
    right.set_xticklabels(right.get_xticklabels(), rotation=rotation)
    
    plt.show()    

def count_lifetime(el):
    '''
    Посчитает количество дней существования стартапа
    '''
    if el['status'] == 'operating':
        return (pd.to_datetime('2018-01-01') - el["founded_at"]).days # самая "правая" дата по условию 2018-01-01
    else:
        return (el['closed_at'] - el["founded_at"]).days

def funding_per_round(value):
    '''
    Подсчитает финансирование за 1 раунд
    '''
    if value['funding_total_usd'] > 0:
        return round(value['funding_total_usd'] / value['funding_rounds'])
    else:
        return 0

def fill_nan_funding(value, funding_avg):
    '''
    Заполнит пропуски в 'funding_total_usd',
    используя величину среднего финансирования за раунд и количество раундов
    '''
    if value['funding_total_usd'] > 0:        
        return value['funding_total_usd']
    else:
        return value['funding_rounds'] * funding_avg

def fill_funding_total_usd(df):
    '''
    Заполнит пропуски в 'funding_total_usd' для датафрейма df
    '''
    df['funding_per_round'] = df.apply(lambda x: funding_per_round(x), axis=1)
    funding_median = df.query('funding_per_round > 0')['funding_per_round'].median()    
    df['funding_total_usd'] = df.apply(lambda x: fill_nan_funding(x, funding_median), axis=1)
    df = df.drop(columns=['funding_per_round'])
    return df    

In [5]:
# инициализация глобальных переменных
RANDOM_STATE = 42 # Deep Thought number
TEST_SIZE = 0.25 # размер выборки

In [6]:
# настройки
pd.options.display.max_columns = None # необходимо чтобы head отобразил все колонки
plt.rcParams['figure.figsize'] = [15, 3] # размеры графика по умолчанию
sns.set(rc={'figure.figsize':(15, 10)}) # размеры графика seaborn по умолчанию
pd.set_option('display.float_format', '{:.2f}'.format) # отобразит в колонках округленные данные до 6-ти знаков

simplefilter(action='ignore', category=FutureWarning) # уберёт предупреждение "Set `keepdims`

# цвета для диаграмм
colors = ['gray','#3672b4','#ac0936']

# Загрузка и изучение данных

## Загрузка

In [15]:
# загрузка данных в датасеты
try:
    apparel_purchases = pd.read_csv('./datasets/apparel-purchases.csv') # история покупок
    apparel_messages = pd.read_csv('./datasets/apparel-messages.csv') # история рекламных рассылок
    apparel_target = pd.read_csv('./datasets/apparel-target_binary.csv')  # целевой признак
except:
    print("Ошибка. Данные не загружены")